In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
from IPython.display import Image
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True' + ',dnn.enabled=False'
import cell2location
os.chdir("/nfs/team283/aa16/SpaceJam/")
import spacejam

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Can not use cuDNN on context None: Disabled by dnn.enabled flag
Mapped name None to device cuda: Tesla V100-SXM2-32GB (0000:89:00.0)


Import our three models:

In [2]:
from spacejam.models import LocationModelWTAMultiExperiment
from spacejam.models import LocationModelWTAMultiExperimentGeneLevel
from spacejam.models import LocationModelWTAMultiExperimentHierarchicalGeneLevel

In [3]:
models = [spacejam.models.LocationModelWTAMultiExperiment.LocationModelWTAMultiExperiment,
          spacejam.models.LocationModelWTAMultiExperimentGeneLevel.LocationModelWTAMultiExperimentGeneLevel,
          spacejam.models.LocationModelWTAMultiExperimentHierarchicalGeneLevel.LocationModelWTAMultiExperimentHierarchicalGeneLevel]

Load and format data:

In [4]:
adata_wta = sc.read_h5ad(open("/lustre/scratch117/cellgen/team283/Kidney-Nanostring/Kidney_AnnData.h5ad", "rb"))

/nfs/users/nfs_a/aa16/.local/lib/python3.7/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
reg_path = "/lustre/scratch117/cellgen/team283/Kidney-Nanostring/scrna/regression_model/RegressionGeneBackgroundCoverageTorch_46covariates_40268cells_10882genes/"
adata_snrna_raw = sc.read(f'{reg_path}sc.h5ad')

In [6]:
# Column name containing cell type annotations
covariate_col_names = 'celltype'

# Extract a pd.DataFrame with signatures from anndata object
inf_aver = adata_snrna_raw.raw.var.copy()
inf_aver = inf_aver.loc[:, [f'means_cov_effect_{covariate_col_names}_{i}' for i in adata_snrna_raw.obs[covariate_col_names].unique()]]
from re import sub
inf_aver.columns = [sub(f'means_cov_effect_{covariate_col_names}_{i}', '', i) for i in adata_snrna_raw.obs[covariate_col_names].unique()]
inf_aver = inf_aver.iloc[:, inf_aver.columns.argsort()]

# normalise by average experiment scaling factor (corrects for sequencing depth)
inf_aver = inf_aver * adata_snrna_raw.uns['regression_mod']['post_sample_means']['sample_scaling'].mean()

In [7]:
counts_negativeProbes = adata_wta.obsm['negProbes']
counts_nuclei = np.asarray(adata_wta.obs['AOINucleiCount']).reshape(len(adata_wta.obs['AOINucleiCount']),1)

In [8]:
adata_wta.raw = adata_wta

In [9]:
inf_aver.index = np.array(adata_snrna_raw.var.loc[inf_aver.index,'SYMBOL'])

Run cell2location:

In [12]:
for i in range(1, len(models)):
    cell2location.run_c2l.run_cell2location(inf_aver, adata_wta, 
            model_name=models[i],
            export_args={'path': "/lustre/scratch117/cellgen/team283/Kidney-Nanostring/results"},
            train_args={'use_raw': True, 'n_iter': 200, 'sample_name_col': 'SlideName'},
            model_kwargs={
            "Y_data" : counts_negativeProbes,
            "cell_number_prior" : {'cells_per_spot': counts_nuclei, 'factors_per_spot': 6, 'combs_per_spot': 3},
            "cell_number_var_prior" : {'cells_mean_var_ratio': 1, 'factors_mean_var_ratio': 1, 'combs_mean_var_ratio': 1}})

### Summarising single cell clusters ###
### Creating model ### - time 0.0 min


ValueError: Input dimension mis-match. (input[0].shape[0] = 9551, input[1].shape[0] = 7)